### Limpieza e inspección de los datos

Este notebook está esturcturado como sigue:

- **Importación de librerias**
- **Bloque A**: carga de los datos.
- **Bloque B**: inspección de los datos.
- **Bloque C**: limpieza y adecuación.
- **Bloque D**: exportación del conjunto resultante.

### Importación librerias

In [56]:
import pandas as pd
import re

import cld3

### BLOQUE A: Carga de los datos

**Pandas**: librería más popular de python que proporciona las herramientas y estructuras necesarias para manipular y analizar datos. La estructura básica de Pandas es el **DataFrame**, una colección ordenada de columnas con nombres y tipos, donde una sola fila representa un único caso (observación) y las columnas representan atributos particulares.

Guía: https://pandas.pydata.org/docs/getting_started/index.html

In [79]:
# Leemos el CSV que contiene los datos y lo cargamos en memoria
df = pd.read_csv("../data/amazon_reviews.csv")

### BLOQUE B: inspección de los datos

El objetivo de la inspección es la familiarización con el conjunto de datos. Algunas preguntas iniciales que podría estar bien hacerse pueden ser:

- ¿En qué tipo de objeto están almacenados los datos?
- ¿Cuál es su dimensión?
- ¿Hay datos ausentes?
- ¿Que tipos diferentes de datos hay?
- ...

In [80]:
# Ver el tipo de objeto con el que estamos trabajando
type(df)

pandas.core.frame.DataFrame

In [81]:
# Dimensiones del dataframe
df.shape

(120000, 3)

In [82]:
# Otra manera más interesante de proporcionar la misma información anterior
print(f"Hay un dataframe de {df.shape[0]} filas y {df.shape[1]} columnas.")

Hay un dataframe de 120000 filas y 3 columnas.


In [83]:
# Resumen conciso del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   class_index   120000 non-null  int64 
 1   review_title  119997 non-null  object
 2   review_text   120000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [84]:
# Mostrar las primeras 5 filas
df.head()

,class_index,review_title,review_text
0,2,It's a Cheap Electric Finish Nailer; It's Not ...,I was given this as a gift after I complained ...
1,2,Frypan.,I've had this pan for about three years.It coo...
2,2,This is why Boorstin is one of my favorites,I love to read American history and Daniel Boo...
3,2,Great Memories,"This is a memorable album, each track will war..."
4,1,This book stinks,Even by adding Mary Higgins Clarks name to thi...


In [85]:
# Mostrar las últimas 5 filas
df.tail()

,class_index,review_title,review_text
119995,1,Width not the same as last pair puchased.,The last pair I got were marked medium width. ...
119996,1,not the original soap,This is not the original unscented ivory. They...
119997,1,Universal Lids,I have the large one tried to use it on coffee...
119998,1,Awkward toy,"I got this toy for my daughter's 1st birthday,..."
119999,2,Gets the job done,I don't think there is a product available tha...


In [86]:
# ¿Cuáles son los posibles valores que puede tomar la variable 'class_index'?
df['class_index'].unique()

array([2, 1])

La variable "class_index" indica el sentimiento de cada reseña:

- 1: sentimiento negativo
- 2: sentimiento positivo

### BLOQUE C: limpieza y adecuación

In [87]:
# Eliminación de las columnas que no son utiles a nuestro estudio
# Nos quedaremos solo con 'Revies'
df = df.drop(columns=['review_title'])
df.head()

,class_index,review_text
0,2,I was given this as a gift after I complained ...
1,2,I've had this pan for about three years.It coo...
2,2,I love to read American history and Daniel Boo...
3,2,"This is a memorable album, each track will war..."
4,1,Even by adding Mary Higgins Clarks name to thi...


In [88]:
""" 
Renombrar las columnas:
    - review_text -> text
    - class_index --> sentiment
"""
df.rename(columns={'review_text': 'text', 'class_index':'sentiment'}, inplace=True)

df.head()

,sentiment,text
0,2,I was given this as a gift after I complained ...
1,2,I've had this pan for about three years.It coo...
2,2,I love to read American history and Daniel Boo...
3,2,"This is a memorable album, each track will war..."
4,1,Even by adding Mary Higgins Clarks name to thi...


In [89]:
# Calcular la longitud de cada reseña
df['length'] = df['text'].apply(len)

In [90]:
# Resumen de la nueva columna "length"
df['length'].describe()

count    120000.000000
mean        404.359558
std         234.059880
min          15.000000
25%         207.000000
50%         355.000000
75%         565.000000
max        1007.000000
Name: length, dtype: float64

In [91]:
# busqueda de los elementos con longitud mínima (15)
df[df.length == 15]

,sentiment,text,length
101991,2,Excellent book.,15


In [92]:
# Vamos a remover aquellos textos que tiene más de 565 caracteres (Q3)
df = df[df.length <= 565]

In [93]:
# Comprobamos las dimensiones actuales del dataframe
df.shape

(90055, 3)

In [94]:
# Detección del idioma en el que está escrita cada reseña
df['language'] = df['text'].apply(lambda x: cld3.get_language(x)[0])

In [95]:
# ¿En que idioma están escritas las reseñas? ¿Cuantas reseñas tenemos para cada idioma?
df['language'].value_counts()

en         89735
es           219
fr            18
pt            10
de             7
it             5
mg             5
pl             4
ca             4
fy             4
so             4
ja             4
cy             4
mt             3
no             2
et             2
nl             2
ja-Latn        2
lb             2
el-Latn        2
zu             1
hi-Latn        1
cs             1
uz             1
hmn            1
sv             1
sl             1
da             1
zh             1
kk             1
ha             1
fil            1
ny             1
af             1
fi             1
gl             1
la             1
Name: language, dtype: int64

In [96]:
# ¿Cual son las reseñas en español?
df[df.language=='es']

,sentiment,text,length,language
489,2,Que no tiene este cd. La electronica-rock lati...,191,es
1809,2,"Marion Cotillard parece encriptada ,fusionada ...",261,es
3226,2,This season of prison break is a great oportun...,468,es
3689,2,Muy buen material. La voz de Edith luce espect...,119,es
3908,2,Paquita es una Dama del bolero del barrio y es...,278,es
...,...,...,...,...
116322,2,Estoy de cauerdo con que Emmanuelle esun gran ...,386,es
117206,1,"Este album es basura, prefiero escuchar mucho ...",175,es
117331,2,"Estimado amigo de la música, en estos momentos...",557,es
119701,1,Grabación en directo. Aunque el precio es bara...,189,es


In [97]:
# Nos quedamos solo con aquellas que están escritas únicamente en inglés
df = df[df['language']=='en']

In [98]:
# Comprobamos las dimensiones actuales del dataframe
df.shape

(89735, 4)

### BLOQUE E: Exportación del conjunto resultante

En esta última fase exportamos y guardamos el conjunto de datos ya limpiado, procesado y adecuado en la carpeta 'data'. Solo guardaremos aquellas columnas que posteriormente nos serán útiles.
Ya que hay bastante datos nos vamo

In [99]:
# Eliminamos las columnas que ya no serán necesarias para los siguiente pasos
df = df.drop(columns=['language'])

In [100]:
# muestreo proporcional
df_sample = df.groupby('sentiment', group_keys=False).apply(
    lambda x: x.sample(frac=0.4, random_state=2023)
)

df_sample.head()

,sentiment,text,length
452,1,This book was extremely well written and gripp...,513
72626,1,1. These wipes are all connected (not folded)....,449
11895,1,If you buy this don't expect to power it with ...,530
15203,1,I didn't think it could get much worse than th...,289
108134,1,1. The information states the hairpins are sil...,468


In [101]:
# vamos a combrobar que las proporcionese se hay mantenido iguales
print("Proporciones antes del muestreo:")
print(df['sentiment'].value_counts(normalize=True), "\n")

print("Proporciones despues del muestreo:")
print(df_sample['sentiment'].value_counts(normalize=True))

Proporciones antes del muestreo:
2    0.511907
1    0.488093
Name: sentiment, dtype: float64 

Proporciones despues del muestreo:
2    0.511896
1    0.488104
Name: sentiment, dtype: float64


In [103]:
# Guardamos los datos ya procesados en un fichero csv
df_sample.to_csv('../data/amazon_reviews_clean.csv', index=False)